## **CODE MẪU CHỈ PHỤC VỤ HỌC TẬP NGHIÊN CỨU TẠI TAPIT, KHÔNG CHIA SẺ THÊM CHO AI NẾU CHƯA ĐƯỢC SỰ ĐỒNG Ý TỪ TAPIT**

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import glob
import os
import math
import random
import numpy as np
root_path = r'/content/drive/MyDrive/Document/EdgeAI/Data/FallDetection'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print(glob.glob(os.path.join(root_path, 'Data/labeled/fall-*-acc.csv')))

['/content/drive/MyDrive/Document/EdgeAI/Data/FallDetection/Data/labeled/fall-01-acc.csv', '/content/drive/MyDrive/Document/EdgeAI/Data/FallDetection/Data/labeled/fall-04-acc.csv', '/content/drive/MyDrive/Document/EdgeAI/Data/FallDetection/Data/labeled/fall-03-acc.csv', '/content/drive/MyDrive/Document/EdgeAI/Data/FallDetection/Data/labeled/fall-05-acc.csv', '/content/drive/MyDrive/Document/EdgeAI/Data/FallDetection/Data/labeled/fall-06-acc.csv', '/content/drive/MyDrive/Document/EdgeAI/Data/FallDetection/Data/labeled/fall-07-acc.csv', '/content/drive/MyDrive/Document/EdgeAI/Data/FallDetection/Data/labeled/fall-08-acc.csv', '/content/drive/MyDrive/Document/EdgeAI/Data/FallDetection/Data/labeled/fall-09-acc.csv', '/content/drive/MyDrive/Document/EdgeAI/Data/FallDetection/Data/labeled/fall-10-acc.csv', '/content/drive/MyDrive/Document/EdgeAI/Data/FallDetection/Data/labeled/fall-11-acc.csv', '/content/drive/MyDrive/Document/EdgeAI/Data/FallDetection/Data/labeled/fall-02-acc.csv', '/content

In [ ]:
df_lst = []
count_lst = []
for file_path in glob.glob(os.path.join(root_path, 'Data/labeled/fall-*-acc.csv')):
  df = pd.read_csv(file_path)
  df_lst.append(df)
  count_lst.append(df.count()['time'])
print(count_lst)

[36, 30, 61, 20, 23, 32, 19, 37, 35, 22, 22, 22, 56, 25, 42, 28, 48, 22, 54, 48, 27, 29, 42, 57, 52, 30, 55, 41, 62, 63]


In [ ]:
frame_length = 15
overlap_ratio = 0.3
hop_length = int(frame_length * (1 - overlap_ratio))
print(hop_length)

10


In [ ]:
df.head()

,time,total,x,z,y
0,-25,1.018869,-0.987793,0.219727,0.118652
1,-17,0.991951,-0.960938,0.219727,0.110840
2,-9,0.979602,-0.945312,0.235352,0.103027
3,-1,0.962084,-0.933594,0.219727,0.075684
4,7,0.933191,-0.902832,0.223633,0.075684


In [ ]:
def feature_raw(slide_df):
  x = slide_df['x'].values.tolist()
  z = slide_df['z'].values.tolist()
  y = slide_df['y'].values.tolist()
  # total = slide_df['total'].values.tolist()
  print(x)
  print(y)
  print(z)
  print(x+y+z)
  print('-------------')
  return x + y + z

In [ ]:
def feature_math(slide_df):
  x_std = slide_df.std()['x']
  y_std = slide_df.std()['y']
  z_std = slide_df.std()['z']

  x_mean = slide_df.mean()['x']
  y_mean = slide_df.mean()['y']
  z_mean = slide_df.mean()['z']
  
  x_skew = slide_df.skew()['x']
  y_skew = slide_df.skew()['y']
  z_skew = slide_df.skew()['z']
  
  return [x_std, y_std, z_std, x_mean, y_mean, z_mean, x_skew, y_skew, z_skew]

In [ ]:
fall_in_lst = []

for df in df_lst:
  clip_length = df.count()['x']
  slide_nums = math.floor(clip_length / hop_length) - 1
  # print(clip_length, '-', slide_nums)
  for i in range(slide_nums):
    idx_min, idx_max = (i*hop_length), (i*hop_length + frame_length)
    slide_df = df[(df.index >= idx_min) & (df.index < idx_max)]
    # fall_in_lst.append(feature_raw(slide_df))    
    fall_in_lst.append(feature_math(slide_df))

1.489313233977418
4.1236150351651615


In [ ]:
print(len(fall_in_lst))

72


In [ ]:
36
15

In [ ]:
adl_in_lst = []
for file_path in glob.glob(os.path.join(root_path, 'Data/raw/adl-*-acc.csv')):
  df = pd.read_csv(file_path, names=['time', 'total', 'x', 'z', 'y'])
  clip_length = df.count()['x']
  slide_nums = math.floor(clip_length / hop_length) - 1
  for i in range(slide_nums):
    idx_min, idx_max = (i*hop_length), (i*hop_length + frame_length)
    slide_df = df[(df.index >= idx_min) & (df.index < idx_max)]
    # input_lst.append(feature_raw(slide_df))
    adl_in_lst.append(feature_math(slide_df))

In [ ]:
print(len(adl_in_lst))

1596


In [ ]:
random.shuffle(adl_in_lst)
adl_in_lst = adl_in_lst[:int(len(fall_in_lst)*1.2)]

In [ ]:
len(adl_in_lst)

86

In [ ]:
adl_in = np.array(adl_in_lst)
adl_lb = np.ones(adl_in.shape[0], dtype=np.uint8)*0
fall_in = np.array(fall_in_lst)
fall_lb = np.ones(fall_in.shape[0], dtype=np.uint8)*1

data_in = np.concatenate((adl_in, fall_in))
data_lb = np.concatenate((adl_lb, fall_lb))

randomize = np.arange(data_in.shape[0])
np.random.shuffle(randomize)
data_in = data_in[randomize]
data_lb = data_lb[randomize]

In [ ]:
print(data_in.shape)
print(data_lb.shape)

(158, 9)
(158,)


In [ ]:
np.save(os.path.join(root_path, 'Data/data_in.npy'), data_in)
np.save(os.path.join(root_path, 'Data/data_lb.npy'), data_lb)